# Определение перспективного тарифа для телеком компании

<b>Описание проекта</b>
<bR>Вы аналитик компании «Мегалайн» — федерального оператора сотовой связи. Клиентам предлагают два тарифных плана: «Смарт» и «Ультра». Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег.</bR>
<bR></bR>
<bR>Вам предстоит сделать предварительный анализ тарифов на небольшой выборке клиентов. В вашем распоряжении данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. 
    
Цель исследования – проанализировать поведение клиентов и сделать вывод, какой тариф лучше.

<b>Шаг 1. Откройте файл с данными и изучите общую информацию</b>

- <b>Информация о звонках

In [ ]:
import pandas as pd
calls = pd.read_csv('/datasets/calls.csv')
calls

In [ ]:
calls.info()

In [ ]:
calls.describe()

Датафрейм содержит 202,6 тысяч строк и 4 столбца. Столбец duration содержит данные не целочисленного типа. Столбец call_date нужно перевести в формат даты. Пропусков нет.

- <b>Информация об интернет-сессиях

In [ ]:
internet = pd.read_csv('/datasets/internet.csv')
internet

In [ ]:
internet.info()

In [ ]:
internet.describe()

Датафрейм содержит 143.4 тысяч строк и 5 столбца. Столбец 'Unnamed: 0' полностью дублирует нумерацию, его можно удалить. Столбец mb_used содержит данные не целочисленного типа. Столбец session_date нужно перевести в формат даты. Пропусков нет.

- <b>Информация о сообщениях

In [ ]:
messages = pd.read_csv('/datasets/messages.csv')
messages

In [ ]:
messages.info()

In [ ]:
messages.describe()

Датафрейм содержит 123 тысяч строк и 3 столбца. Столбец message_date нужно перевести в формат даты. Пропусков нет.


- <b>Информация о тарифах

In [ ]:
tariffs = pd.read_csv('/datasets/tariffs.csv')
tariffs

In [ ]:
tariffs.info()

In [ ]:
tariffs.describe()

Датафрейм содержит информацию из 2 тарифов. Пропусков нет, все данные в нужном формате.

- <b>Информация о пользователях

In [ ]:
user_info = pd.read_csv('/datasets/users.csv')
user_info

In [ ]:
user_info.info()

In [ ]:
user_info.describe()

Датафрейм содержит 500 строк и 8 столбцов. Столбецы churn_date и reg_date нужно перевести в формат даты. Пропусков нет.


<b>Шаг 2. Подготовьте данные

- Приведите данные к нужным типам

In [ ]:
#перевод в формат даты
from datetime import datetime

calls['call_date'] = pd.to_datetime(calls.call_date, format= '%Y/%m/%d')
internet['session_date'] = pd.to_datetime(internet.session_date, format= '%Y/%m/%d')
messages['message_date'] = pd.to_datetime(messages.message_date, format= '%Y/%m/%d')
user_info['churn_date'] = pd.to_datetime(user_info.churn_date, format= '%Y/%m/%d')
user_info['reg_date'] = pd.to_datetime(user_info.reg_date, format= '%Y/%m/%d')

In [ ]:
#перевод данных в целочисленный тип
calls['duration'] = calls['duration'].astype('int')
internet['mb_used'] = internet['mb_used'].astype('int')

- Найдите и исправьте ошибки в данных

In [ ]:
#округление минут и мегабайтов вверх 
import numpy as np

calls['duration'] = calls['duration'].apply(np.ceil)
internet['mb_used'] = internet['mb_used'].apply(np.ceil)

In [ ]:
calls['duration'].describe().round()

In [ ]:
internet['mb_used'].describe().round()

In [ ]:
#удаление дублирующего столбца в таблице internet
del internet['Unnamed: 0']
internet

<b>Посчитайте для каждого пользователя:

- Количество сделанных звонков и израсходованных минут разговора по месяцам

In [ ]:
calls['month'] = calls['call_date'].dt.month_name()
calls['month'].value_counts()

In [ ]:
calls_per_months = calls.groupby(['user_id', 'month'])\
                  .agg({'duration':'sum', 'id':'count'})\
                  .reset_index()\
                  .rename(columns={'duration':'talk_time','id':'calls_per_month'})\

calls_per_months

In [ ]:
calls_per_months.describe()

В среднем каждый пользователь делает по 63 звонка общей длительностью 396 минут ежемесячно. То есть 1 звонок длится в среднем 6,3 минуты.

- Количество отправленных сообщений по месяцам

In [ ]:
messages['month'] = messages['message_date'].dt.month_name()
messages['month'].value_counts()

In [ ]:
messages_per_months = messages.groupby(['user_id', 'month'])\
                  .agg({'id':'count'})\
                  .reset_index()\
                  .rename(columns={'id':'sms_per_month'})\

messages_per_months

In [ ]:
messages_per_months.describe()

Каждый пользователь в среднем отправляет 38 сообщений в месяц.

- Объем израсходованного интернет-трафика по месяцам

In [ ]:
internet['month'] = internet['session_date'].dt.month_name()
internet['month'].value_counts()

In [ ]:
internet_per_months = internet.groupby(['user_id', 'month'])\
                  .agg({'mb_used':'count'})\
                  .reset_index()\
                  .rename(columns={'mb_used':'mb_per_month'})\

internet_per_months

In [ ]:
internet_per_months.describe()

В среднем каждый пользователь использует 48 мб трафика ежемесячно.

- Помесячная выручка с каждого пользователя

In [ ]:
#для подсчета помесячной выручки лучше объединить таблицы
data = calls_per_months.merge(messages_per_months, on=['user_id','month'], how='outer')
data

In [ ]:
data = data.merge(internet_per_months, on=['user_id','month'], how='outer')
data

In [ ]:
data = data.merge(user_info, on='user_id', how='left')
data

In [ ]:
data.rename(columns=lambda x: x.replace('tariff', 'tariff_name'), inplace=True)
data

In [ ]:
data = data.merge(tariffs, on='tariff_name', how='left')
data

Задание: вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана; прибавьте абонентскую плату, соответствующую тарифному плану

In [ ]:
#Заменяем отсутствующие значения на нули
data['talk_time'].fillna(0, inplace=True)
data['calls_per_month'].fillna(0, inplace=True)
data['sms_per_month'].fillna(0, inplace=True)
data['mb_per_month'].fillna(0, inplace=True)

data

In [ ]:
#Высчитываем количество минут сверх тарифа 
data['extra_minutes'] = data['talk_time'] - data['minutes_included']
data['extra_messeges'] = data['sms_per_month'] - data['messages_included']
data['extra_gb'] = np.floor((data['mb_per_month'] - data['mb_per_month_included']))/1024
data

In [ ]:
#убираем отрицательные значения в сверх минутах, смс и гб 

data.loc[data['extra_minutes'] <0, 'extra_minutes'] = 0
data.loc[data['extra_messeges'] <0, 'extra_messeges'] = 0
data.loc[data['extra_gb'] <0, 'extra_gb'] = 0
data

In [ ]:
data['total_payment'] = data['rub_monthly_fee'] + data['extra_minutes']*data['rub_per_minute'] + data['extra_messeges']*data['rub_per_message'] + data['extra_gb']*data['rub_per_gb']
data.sort_values(by='total_payment', ascending=False)

In [ ]:
payment_month = data.pivot_table(index='month', values='total_payment', aggfunc=['sum','count'])

payment_month

In [ ]:
data['total_payment'].sum()

<b> Вывод

Обработаны таблицы с информацией о пользователях, их интернет-трафике, отправленных сообщениях, звонках. Для каждого пользователя были количество сделанных звонков и израсходованных минут разговора, количество отправленных сообщений, расходы на сотовую связь ежемесячно. Все изначальные таблицы были объединены в одну. 
<bR>Ежемесячно прибыль растет. Общая прибыль за 2018 год составляет 3 331 185 руб. </bR>
<bR>В среднем каждый пользователь ежемесячно:
    - делает по 63 звонка общей длительностью 396 минут (1 звонок равен в среднем 6,3 минутам).
    - отправляет 38 сообщений в месяц.
    - использует 48 мб трафика ежемесячно.

<b>Шаг 3. Проанализируйте данные

Опишите поведение клиентов оператора, исходя из выборки. Сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц? Посчитайте среднее количество, дисперсию и стандартное отклонение. Постройте гистограммы. Опишите распределения.

In [ ]:
data['tariff_name'].value_counts()

In [ ]:
data.pivot_table(index='tariff_name',values='total_payment', aggfunc='sum')

In [ ]:
(data
    .pivot_table(index='tariff_name',values='total_payment', aggfunc='median')
    .plot(kind='bar', grid=True, title='Средняя сумма платежа по тарифому плану')
)

In [ ]:
data['add_payment'] = data['extra_minutes']*data['rub_per_minute'] + data['extra_messeges']*data['rub_per_message'] + data['extra_gb']*data['rub_per_gb']
data['add_payment'].value_counts()


In [ ]:
data.pivot_table(index='tariff_name',values='add_payment', aggfunc='sum')

In [ ]:
data[data['tariff_name'] == 'smart']['total_payment'].plot(kind='hist', title='Платежи по тарифу смарт')

In [ ]:
data[data['tariff_name'] == 'ultra']['total_payment'].plot(kind='hist', title='Платежи по тарифу ультра')

In [ ]:
import numpy as np

data.groupby('tariff_name')\
  .agg({'total_payment':['median','mean','var','std', 'max']})\
  .style.format("{:.2f}")

Клиентам, подключенным к тарифу Ультра, всегда хватает предложенного количества смс, звонков и интернета. Клиенты, подключенные к тарифу Смарт, иногда оплачивают смс, звонки и интернет дополнительно.

- Сколько минут разговора требуется пользователям каждого тарифа в месяц

In [ ]:
data[data['tariff_name'] == 'smart']['talk_time'].plot(kind='hist', title='Количество минут по тарифу Смарт')

Данные скошены в отрицательную сторону. Медиана будет больше средней.

In [ ]:
data[data['tariff_name'] == 'ultra']['talk_time'].plot(kind='hist', title='Количество минут по тарифу Ультра')

Данные скошены в положительную сторону. Медиана будет меньше средней.

In [ ]:
data.groupby('tariff_name')\
  .agg({'talk_time':['median','mean','var','std', 'max']})\
  .style.format("{:.2f}")

Согласно графику пользователи тарифа Смарт, имея ограничение в 500 минут ежемесячно, иногда выходят за его пределы. Кто-то даже превысил ограничение в 2,5 раза. В среднем клиент с тарифом Смарт разговаривает 375 минут в месяц. <bR>
Клиенты, подключенные к тарифу Ультра, имеют ограничение в 3000 минут ежемесячно. Из предложенной выборки ни один клиент не использовал доступный пакет минут в полном объеме. Максимальное значение – 1484 минуты. В среднем клиент тарифа Ультра разговаривает 458 минут в месяц. 

- Сколько сообщений требуется пользователям каждого тарифа в месяц? 

In [ ]:
data[data['tariff_name'] == 'smart']['sms_per_month'].plot(kind='hist', title='Количество смс по тарифу Смарт')

In [ ]:
data[data['tariff_name'] == 'ultra']['sms_per_month'].plot(kind='hist', title='Количество смс по тарифу Ультра')

In [ ]:
data.groupby('tariff_name')\
  .agg({'sms_per_month':['median','mean','var','std', 'max']})\
  .style.format("{:.2f}")

Оба графика, отражающих количество смс, скошены в положительную сторону. <bR> Согласно графику пользователи тарифа Смарт, имея ограничение в 50 сообщений ежемесячно, иногда выходят за его пределы. В среднем клиент с тарифом Смарт отправляет 28 сообщений в месяц.</bR>
<bR>Клиенты, подключенные к тарифу Ультра, имеют ограничение в 1000 смс ежемесячно. Из предложенной выборки ни один клиент не использовал доступный пакет смс в полном объеме. Максимальное значение – 224 смс. В среднем клиент тарифа Ультра отправляет 38 сообщений в месяц.


- Какой объём интернет-трафика требуется пользователям каждого тарифа в месяц

In [ ]:
data[data['tariff_name'] == 'smart']['mb_per_month'].plot(kind='hist', title='Интернет-трафик по тарифу Смарт (мб)')

In [ ]:
data[data['tariff_name'] == 'ultra']['mb_per_month'].plot(kind='hist', title='Интернет-трафик по тарифу Ультра (мб)')

In [ ]:
data.groupby('tariff_name')\
  .agg({'mb_per_month':['median','mean','var','std', 'max']})\
  .style.format("{:.2f}")

График использования интернета у клиентов тарифа Смарт скошен влево, у клиентов тарифа Ультра – вправо. <bR> Клиенты, подключенные к тарифу Смарт, пользуются интернетом больше, чем клиенты тарифа Ультра. Средний размер интернет-трафика в тарифе Смарт составляет 51 мб в месяц, в тарифе Ультра – 38 мб в месяц. Нет никого, кто бы истратил весь свой ежемесячный интернет-трафик.

<b>Шаг 4. Проверьте гипотезы

Поскольку выборки независимы друг от друга, для проверки гипотез используется тест Стьюдента.

- Cредняя выручка пользователей тарифов «Ультра» и «Смарт» различается

H0 - Cредняя выручка пользователей тарифов «Ультра» и «Смарт» не различается <bR>
H1 – Cредняя выручка пользователей тарифов «Ультра» и «Смарт» различается

In [ ]:
ultra_profit = data[data['tariff_name'] == 'ultra']
smart_profit = data[data['tariff_name'] == 'smart']

In [ ]:
from scipy import stats as st

ultra_profit = ultra_profit['total_payment'] 
smart_profit = smart_profit['total_payment'] 

alpha = 0.05

results = st.ttest_ind(
    ultra_profit,
    smart_profit) 

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Исходя из результатов t-теста средняя выручка пользователей тарифов «Ультра» и «Смарт» различается.

- Cредняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов

H0 - Cредняя выручка пользователей из Москвы не отличается от выручки пользователей из других регионов <bR>
H1 – Cредняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов

In [ ]:
data['city'].unique()

In [ ]:
moscow = data[data['city'] == 'Москва']
other_regions = data.query('not city == "Москва"')
other_regions['city'].unique()

In [ ]:
moscow_profit = moscow['total_payment']
other_regions_profit = other_regions['total_payment'] 

alpha = 0.05

results = st.ttest_ind(
    moscow_profit,
    other_regions_profit) 

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Исходя из результатов t-теста средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов.

<b>Шаг 5. Напишите общий вывод

Ежемесячно прибыль компании растет. Общая прибыль за 2018 год составляет 3 331 185 руб. <bR>
Доходы тарифа Смарт в 2018 году составили 1,4 млн. рублей, от тарифа Ультра – 1,9 млн. рублей. Пользователи тарифа Смарт регулярно выходят за лимиты по количеству смс, минут разговора и интернет-трафику и доплачивают за это, в то время как пользователи тарифа Ультра чаще всего вообще не достигают этого лимита. <bR>
В целом клиенты обоих тарифов разговаривают по телефону до 500 минут в месяц. Клиенты тарифа Смарт разговаривают немного меньше.</bR>
<bR> Проведение теста Стьюдента подтвердила, что средняя выручка пользователей из Москвы и средняя выручка пользователей из других регионов не равны.</bR> 
<bR> Стоит пересмотреть тарифы – немного расширить границы для тарифа Смарт и сократить лимит для тарифа Ультра.</bR> 
<bR> С точки зрения прибыли, лучшим тарифом является Ультра.